In [1]:
###################################################################################################
#
# PairIdentification.py
#
# Copyright (C) by Andreas Zoglauer & Harrison Costatino.
#
# Please see the file LICENSE in the main repository for the copyright-notice.
#
###################################################################################################



###################################################################################################

import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

import numpy as np

#from mpl_toolkits.mplot3d import Axes3D
#import matplotlib.pyplot as plt

import random

import signal
import sys
import time
import math
import csv
import os
import argparse
from datetime import datetime
from functools import reduce


print("\nPair Identification")
print("============================\n")



# Step 1: Input parameters
###################################################################################################


# Default parameters

UseToyModel = True

# Split between training and testing data
TestingTrainingSplit = 0.1

MaxEvents = 10

# File names
FileName = "PairIdentification.p1.sim.gz"
GeometryName = "$(MEGALIB)/resource/examples/geomega/GRIPS/GRIPS.geo.setup"


# Set in stone later
TestingTrainingSplit = 0.8

OutputDirectory = "Results"


parser = argparse.ArgumentParser(description='Perform training and/or testing of the pair identification machine learning tools.')
parser.add_argument('-f', '--filename', default='PairIdentification.p1.sim.gz', help='File name used for training/testing')
parser.add_argument('-m', '--maxevents', default='1000', help='Maximum number of events to use')
parser.add_argument('-s', '--testingtrainigsplit', default='0.1', help='Testing-training split')
parser.add_argument('-b', '--batchsize', default='128', help='Batch size')

args = parser.parse_args()

if args.filename != "":
  FileName = args.filename

if int(args.maxevents) > 1000:
  MaxEvents = int(args.maxevents)

if int(args.batchsize) >= 16:
  BatchSize = int(args.batchsize)

if float(args.testingtrainigsplit) >= 0.05:
  TestingTrainingSplit = float(args.testingtrainigsplit)


if os.path.exists(OutputDirectory):
  Now = datetime.now()
  OutputDirectory += Now.strftime("_%Y%m%d_%H%M%S")

os.makedirs(OutputDirectory)



###################################################################################################
# Step 2: Global functions
###################################################################################################


# Take care of Ctrl-C
Interrupted = False
NInterrupts = 0
def signal_handler(signal, frame):
  global Interrupted
  Interrupted = True
  global NInterrupts
  NInterrupts += 1
  if NInterrupts >= 2:
    print("Aborting!")
    sys.exit(0)
  print("You pressed Ctrl+C - waiting for graceful abort, or press  Ctrl-C again, for quick exit.")
signal.signal(signal.SIGINT, signal_handler)


# Everything ROOT related can only be loaded here otherwise it interferes with the argparse
from EventData import EventData

# Load MEGAlib into ROOT so that it is usable
import ROOT as M
M.gSystem.Load("$(MEGALIB)/lib/libMEGAlib.so")
M.PyConfig.IgnoreCommandLineOptions = True



###################################################################################################
# Step 3: Create some training, test & verification data sets
###################################################################################################


# Read the simulation file data:
DataSets = []
NumberOfDataSets = 0

if UseToyModel == True:
  for e in range(0, MaxEvents):
    Data = EventData()
    Data.createFromToyModel(e)
    DataSets.append(Data)
    
    NumberOfDataSets += 1
    if NumberOfDataSets > 0 and NumberOfDataSets % 1000 == 0:
      print("Data sets processed: {}".format(NumberOfDataSets))
  
else:
  # Load geometry:
  Geometry = M.MDGeometryQuest()
  if Geometry.ScanSetupFile(M.MString(GeometryName)) == True:
    print("Geometry " + GeometryName + " loaded!")
  else:
    print("Unable to load geometry " + GeometryName + " - Aborting!")
    quit()


  Reader = M.MFileEventsSim(Geometry)
  if Reader.Open(M.MString(FileName)) == False:
    print("Unable to open file " + FileName + ". Aborting!")
    quit()


  print("\n\nStarted reading data sets")
  NumberOfDataSets = 0
  while NumberOfDataSets < MaxEvents:
    Event = Reader.GetNextEvent()
    if not Event:
      break

    if Event.GetNIAs() > 0:
      Data = EventData()
      if Data.parse(Event) == True:
        if Data.hasHitsOutside(XMin, XMax, YMin, YMax, ZMin, ZMax) == False:
          DataSets.append(Data)
          NumberOfDataSets += 1
          if NumberOfDataSets % 500 == 0:
            print("Data sets processed: {}".format(NumberOfDataSets))

print("Info: Parsed {} events".format(NumberOfDataSets))

# Split the data sets in training and testing data sets

TestingTrainingSplit = 0.75


numEvents = len(DataSets)

numTraining = int(numEvents * TestingTrainingSplit)

TrainingDataSets = DataSets[:numTraining]
TestingDataSets = DataSets[numTraining:]



# For testing/validation split
# ValidationDataSets = TestingDataSets[:int(len(TestingDataSets)/2)]
# TestingDataSets = TestingDataSets[int(len(TestingDataSets)/2):]

print("###### Data Split ########")
print("Training/Testing Split: {}".format(TestingTrainingSplit))
print("Total Data: {}, Training Data: {},Testing Data: {}".format(numEvents, len(TrainingDataSets), len(TestingDataSets)))
print("##########################")


###################################################################################################
# Step 4: Setting up the neural network
###################################################################################################



###################################################################################################
# Step 5: Training and evaluating the network
###################################################################################################



Pair Identification

Welcome to JupyROOT 6.18/04
Start: -0.08236182240139112, 13.800899578994773, 6
Event ID: 0
  Origin Z: 6
  Gamma Energy: 10000.0
  Hit 1 (origin: 0): type=m, pos=(-0.08236182240139112, 13.800899578994773, 6.0)cm, E=536.4966469728458keV
  Hit 2 (origin: 1): type=e, pos=(0.3442961395754284, 14.318640794288454, 5.0)cm, E=615.8028474138482keV
  Hit 3 (origin: 2): type=e, pos=(0.5780907102184861, 14.770114509178857, 4.0)cm, E=674.1128404229287keV
  Hit 4 (origin: 3): type=e, pos=(-0.06497145924042402, 14.891860489713896, 3.0)cm, E=703.6128967746579keV
  Hit 5 (origin: 4): type=e, pos=(1.3089920606767436, 18.35717540005171, 2.0)cm, E=768.6264544941008keV
  Hit 6 (origin: 5): type=e, pos=(-0.6360896370511371, 17.69607527069661, 1.0)cm, E=809.9152945540886keV
  Hit 7 (origin: 6): type=e, pos=(0.5716051716209518, 15.937007019945932, 2.0)cm, E=862.5868551871705keV
  Hit 8 (origin: 7): type=e, pos=(0.9934312987954474, 20.171129132901946, 1.0)cm, E=910.2409202792309keV
  Hit 

In [2]:
def generate_incidence(edges, pos_data):
    #Generate Incidence Matrix from Edge List
    n_hits = len(pos_data)
    n_edges = len(edges)
    Ri = np.zeros((n_hits, n_edges), dtype=np.uint8)
    Ro = np.zeros((n_hits, n_edges), dtype=np.uint8)
    
    for i in range(len(edges)):
        point = edges[i]
        from_pt = point[0]
        to_pt = point[1]
        Ro[from_pt][i] = 1
        Ri[to_pt][i] = 1
    
    return Ri, Ro

In [3]:
def connect_pos(pos_data):
    #Manually Connect Graph based on Positions
    edges = []
    for i in range(len(pos_data)):
        point_A = pos_data[i]
        z_A = point_A[2]

        for j in range(len(pos_data)):
            point_B = pos_data[j]
            z_B = point_B[2]

            if z_B == z_A + 1:
                edges.append((i, j))
                edges.append((j, i))
    return generate_incidence(edges, pos_data)

In [4]:
def pad(arr, shape):
    #Padd arr to Shape
    padded_arr = np.zeros(shape)
    if len(shape) == 1:
        padded_arr[:arr.shape[0]] = arr
    elif len(shape) == 2:
        padded_arr[:arr.shape[0],:arr.shape[1]] = arr
    return padded_arr

In [74]:
def vectorize_data(eventArr):
    # Edge Validity Labels, Manually Connected Rin, Mannually Connected Rout, XYZ, Type, Energy, Gamma Energy
    Edge_Labels, True_Ri, True_Ro, Man_Ri, Man_Ro, XYZ, Type, Energy, GammaEnergy = [], [], [], [], [], [], [], [], []
    max_hits, max_edges = 0, 0
    
    #Start Parsing Events
    for event in eventArr:
        #Keep track of max hits for padding
        max_hits = max(max_hits, len(event.X))
        
        #Generate Incidence Matrices based on Edges
        edges = []
        pos = np.swapaxes(np.vstack((event.X, event.Y, event.Z)), 0, 1)
        for i in range(1,len(event.Origin+1)):
            edges.append((event.Origin[i-1]-1,i-1))
        print(edges)
        e_Ri, e_Ro = generate_incidence(edges,pos)
        
        #Generate Proposed Incidence Matrices based on Positions
        p_Ri, p_Ro = connect_pos(pos)
        
        #Generate Edge Labels (0 - fake edge; 1 - true edge)
        e_label = np.zeros(p_Ri.shape[1])
        for i in range(p_Ri.shape[1]):
            out = np.where(p_Ro[:,i] == 1)[0][0]
            inn = np.where(p_Ri[:,i] == 1)[0][0]
            e_label[i] = 1*((out, inn) in edges)
            
        #Keep track of max edges for Padding
        max_edges = max(max_edges, p_Ri.shape[1])
        
        #Add all of the event data to lists
        Edge_Labels.append(e_label)
        True_Ri.append(e_Ri)
        True_Ro.append(e_Ro)
        Man_Ri.append(p_Ri)
        Man_Ro.append(p_Ro)
        XYZ.append(np.vstack((event.X, event.Y, event.Z)).T)
        Type.append(2*(event.Type=='m')+(event.Type=='p'))
        Energy.append(event.E)
        GammaEnergy.append(event.GammaEnergy)
    
    #Padding based on Max Hits and Max Edges
    for i in range(len(Edge_Labels)):
        Edge_Labels[i] = pad(Edge_Labels[i],(max_edges,))
        Man_Ri[i] = pad(Man_Ri[i],(max_hits,max_edges))
        Man_Ro[i] = pad(Man_Ro[i],(max_hits,max_edges))
        XYZ[i] = pad(XYZ[i],(max_hits,3))
        Type[i] = pad(Type[i],(max_hits,))
        Energy[i] = pad(Energy[i],(max_hits,))
    
    return np.array(Edge_Labels, dtype=np.float32), np.array(Man_Ri, dtype=np.float32), np.array(Man_Ro, dtype=np.float32), np.array(XYZ, dtype=np.float32), np.array(Type, dtype=np.float32), np.array(Energy, dtype=np.float32), np.array(GammaEnergy, dtype=np.float32)

In [75]:
Edge_Labels, Man_Ri, Man_Ro, XYZ, Type, Energy, GammaEnergy = vectorize_data(TrainingDataSets)

[(-1, 0), (0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 6), (6, 7), (7, 8), (0, 9), (9, 10), (10, 11)]
[(-1, 0), (0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 6), (6, 7), (7, 8), (8, 9), (9, 10), (10, 11), (11, 12), (12, 13), (13, 14), (14, 15)]
[(-1, 0), (0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 6), (0, 7), (7, 8), (8, 9), (9, 10), (10, 11), (11, 12)]
[(-1, 0), (0, 1), (1, 2), (0, 3), (3, 4), (4, 5), (5, 6), (6, 7), (7, 8), (8, 9), (9, 10), (10, 11), (11, 12)]
[(-1, 0), (0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 6), (6, 7), (0, 8), (8, 9), (9, 10), (10, 11)]
[(-1, 0), (0, 1), (1, 2), (2, 3), (3, 4), (0, 5), (5, 6), (6, 7), (7, 8), (8, 9), (9, 10), (10, 11), (11, 12)]
[(-1, 0), (0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (0, 6), (6, 7), (7, 8), (8, 9), (9, 10)]


In [76]:
Features = [[XYZ[i],Man_Ri[i], Man_Ro[i]] for i in range(XYZ.shape[0])]
Labels = Edge_Labels

In [77]:
import torch.distributed as dist
from torch.utils.data import DataLoader
from torch.utils.data.distributed import DistributedSampler

# Locals
from datasets import get_data_loaders
from trainers import get_trainer

In [78]:
train_dataset = [[Features[i],Labels[i]] for i in range(XYZ.shape[0])]

In [79]:
for i, (batch_input, batch_target) in enumerate(train_dataset):
    print(i)

0
1
2
3
4
5
6


In [81]:
trainer = get_trainer('gnn')
trainer.build_model(input_dim=3)
trainer.print_model_summary()

In [82]:
train_data_loader = DataLoader(train_dataset, batch_size=1)
test_data_loader = DataLoader(train_dataset, batch_size=1)

In [83]:
summary = trainer.train(train_data_loader=train_data_loader,
                        valid_data_loader=test_data_loader, n_epochs = 10)

batch_input
[tensor([[[-0.0824, 13.8009,  6.0000],
         [ 0.3443, 14.3186,  5.0000],
         [ 0.5781, 14.7701,  4.0000],
         [-0.0650, 14.8919,  3.0000],
         [ 1.3090, 18.3572,  2.0000],
         [-0.6361, 17.6961,  1.0000],
         [ 0.5716, 15.9370,  2.0000],
         [ 0.9934, 20.1711,  1.0000],
         [-4.4576, 20.0027,  0.0000],
         [-1.1022, 13.5804,  5.0000],
         [-5.4075,  9.3570,  6.0000],
         [-6.1947,  8.6743,  7.0000],
         [-6.3530,  7.8709,  8.0000],
         [ 0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000]]]), tensor([[[0., 1., 1.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]]), tensor([[[1., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 1.,  ..., 0., 0.

batch_input
[tensor([[[  1.6639,  10.8717,  12.0000],
         [  1.2307,  11.0152,  11.0000],
         [  1.8653,  13.0118,  10.0000],
         [  1.2090,  14.6380,   9.0000],
         [-17.6812,   6.8577,   8.0000],
         [-17.3193,   6.0775,   9.0000],
         [ -9.2703,  16.1247,  10.0000],
         [ -9.0947,  14.6796,   9.0000],
         [  1.9630,   9.3010,  11.0000],
         [  3.6469,   8.2704,  10.0000],
         [  4.1611,   8.3854,   9.0000],
         [  3.5100,   8.8034,   8.0000],
         [  7.6308,   8.4351,   9.0000],
         [  0.0000,   0.0000,   0.0000],
         [  0.0000,   0.0000,   0.0000],
         [  0.0000,   0.0000,   0.0000],
         [  0.0000,   0.0000,   0.0000]]]), tensor([[[1., 0., 0.,  ..., 0., 0., 0.],
         [0., 1., 1.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]]), tensor([[[0., 1., 0.,  .

batch_input
[tensor([[[  9.1895,  10.9237,  17.0000],
         [  7.6118,  10.5163,  16.0000],
         [  8.3496,   7.0027,  17.0000],
         [ 80.2521, 292.8224,  16.0000],
         [ 75.6839, 292.9004,  15.0000],
         [ 75.6709, 292.1913,  16.0000],
         [  9.4770,  10.6030,  16.0000],
         [  8.8687,  10.6875,  15.0000],
         [  5.7170,  15.3177,  14.0000],
         [  7.7209,  13.5269,  15.0000],
         [  5.6742,  13.5360,  14.0000],
         [  6.7225,  13.0919,  15.0000],
         [  0.0000,   0.0000,   0.0000],
         [  0.0000,   0.0000,   0.0000],
         [  0.0000,   0.0000,   0.0000],
         [  0.0000,   0.0000,   0.0000],
         [  0.0000,   0.0000,   0.0000]]]), tensor([[[1., 0., 0.,  ..., 0., 0., 0.],
         [0., 1., 0.,  ..., 0., 0., 0.],
         [0., 0., 1.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]]), tensor([[[0., 1., 0.,  .

In [137]:
XYZ[0]

array([[  3.85381919,   6.44699536,  -7.        ],
       [  4.00080172,   6.44560929,  -8.        ],
       [  4.09291421,   6.4434291 ,  -9.        ],
       [  4.2466064 ,   6.31060886, -10.        ],
       [  4.36460104,   6.52443449, -11.        ],
       [  4.42223393,   6.90297795, -12.        ],
       [  4.60253559,   7.50021234, -13.        ],
       [  4.16202441,   8.11827234, -14.        ],
       [  4.82566545,  10.1794609 , -15.        ],
       [  3.72365243,   8.82462364, -16.        ],
       [  8.95757892,  20.92838955, -15.        ],
       [  6.81184758,  18.17600153, -16.        ],
       [  9.25953755,  17.88431744, -15.        ],
       [  9.6855503 ,  18.82045685, -16.        ],
       [  1.52745594,   5.78874964,  -8.        ],
       [  1.91741115,   6.67391267,  -7.        ],
       [  0.        ,   0.        ,   0.        ]])

In [138]:
Man_Ri[0]

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [139]:
Man_Ro[0]

array([[0., 1., 0., ..., 0., 0., 0.],
       [1., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [140]:
Edge_Labels[0]

array([0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 1., 0., 1., 0.,
       1., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0.,
       0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [116]:
a = np.array([[1,2,6],[4,5,6]])

In [121]:
np.where(a[:,2] == 6)[0][0]

0

In [118]:
a[:,2]

array([6, 6])